# Offline Tuple Packing for Graphs to reduce padding

When processing a batch of graphs in machine learning models, it is common 
to combine (pack) several small graphs into one
overall graph to accelerate processing and reduce the overhead of padding.
This tutorial guides you through some data analysis on Graphs and graph packing to determine the efficient strategies for processing datasets drawn out of the OGB competition.

It was shown in "Packing: Towards 2x NLP BERT Acceleration. arXiv, jun 2021." by Matej Kosec et al. 2021 that for transformers padding can be a major part of the data and can cause a slow down of 50% and more. 
As a consequence, the distribution of sizes and degrees of graphs in datasets and how much padding they require, is an important consideration.

This tutorial reproduces the results from the paper "Tuple Packing: Efficient Batching of Small Graphs in Graph Neural Networks" available on [arxiv](https://arxiv.org/abs/2209.06354).

## Analysing graph sizes

For the graph analysis, we focus on datasets in the OGB challenge that we load into PyTorch Geometric.

In [ ]:
import os
import ogb
from ogb.graphproppred import PygGraphPropPredDataset
from torch_geometric.loader import DataLoader
from ogb.utils.mol import smiles2graph

ogb.utils.smiles2graph = smiles2graph
from ogb.lsc.pcqm4mv2_pyg import PygPCQM4Mv2Dataset
from pprint import pprint


This first step loads those datasets ready for additional processing:

In [ ]:
ogb_data = {}


def get_training_dataloader(d_name: str) -> DataLoader:
    if d_name == "ogbg-pcqm4mv2":
        dataset = PygPCQM4Mv2Dataset(smiles2graph=smiles2graph)
    else:
        dataset = PygGraphPropPredDataset(name=d_name)

    split_idx = dataset.get_idx_split()
    train_loader = DataLoader(dataset[split_idx["train"]], batch_size=1, shuffle=False)
    return train_loader


for key in ["ogbg-molhiv", "ogbg-molpcba", "ogbg-code2", "ogbg-pcqm4mv2", "ogbg-ppa"]:
    print("loading:", key)
    ogb_data[key] = get_training_dataloader(key)

We are interested in analysing the distribution in the number of edges and the number of nodes which is observed in the OGB small graph datasets.
For this analysis, we do not need the graphs, we can limit ourselves to 2D histograms that map a number of graphs to tuples with the number of nodes and edges to get a sense of the size distribution of the data.

Calculating the histograms can be a slow process so the results are cached to make running subsequent analysis faster.

In [ ]:
from collections import defaultdict
import pickle
from tqdm import tqdm


def get_histogram(data_loader):
    histogram = defaultdict(int)
    for item in tqdm(data_loader):
        histogram[(item.num_nodes, item.num_edges)] += 1
    return histogram


load_histogram = True
histogram_file = "histograms.pkl"

if os.path.exists(histogram_file) and load_histogram:
    # Generating the histograms takes ~30mn so we load a saved version
    # of the histograms if it has already been generated
    with open(histogram_file, "rb") as f:
        ogb_histograms = pickle.load(f)
else:
    ogb_histograms = {}
    for key in ogb_data:
        print("creating histogram:", key)
        ogb_histograms[key] = get_histogram(ogb_data[key])

# Save the histogram after processing
with open(histogram_file, "wb") as f:
    pickle.dump(ogb_histograms, f)

print("Displaying Histogram for 'ogbg-molhiv'")
print(ogb_histograms["ogbg-molhiv"])

Let's first get some basic stats on the datasets:

In [ ]:
def get_num_samples(histogram):
    return sum([histogram[key] for key in histogram])


print("Number of graphs in each dataset:")
for key in ogb_histograms:
    print(key, get_num_samples(ogb_histograms[key]))

In [ ]:
def get_max_tuples_length(histogram):
    """Get the maximum entry size for each tuple component"""
    maximum_length = []
    for key in histogram:
        if not maximum_length:
            maximum_length = list(key)
        for index, entry in enumerate(maximum_length):
            maximum_length[index] = max(entry, key[index])
    return maximum_length


# getting  max_tuples_length
ogbg_mtl_dict = {}
for key in ogb_histograms:
    ogbg_mtl_dict[key] = get_max_tuples_length(ogb_histograms[key])
print("Maximum number of nodes and edges in each graph:")
pprint(ogbg_mtl_dict)

Detailed statistics on padding efficiency will be provided later in the tutorial.

For now, let's explore the distribution of number of nodes and edges in each of the datasets:

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
from pathlib import Path

figure_directory = Path("graph_packing_tutorial")
figure_directory.mkdir(parents=True, exist_ok=True)


def visualize_2D_histogram(histogram, key, dropout=0.01):
    total_count = sum([histogram[(nodes, edges)] for nodes, edges in histogram])
    threshold = total_count * dropout / 100
    num_nodes = [
        nodes for nodes, edges in histogram if histogram[(nodes, edges)] >= threshold
    ]
    num_edges = [
        edges for nodes, edges in histogram if histogram[(nodes, edges)] >= threshold
    ]
    image = np.zeros([max(num_nodes) + 1, max(num_edges) + 1])
    exceptions = []
    for nodes, edges in histogram:
        try:
            image[nodes][edges] = histogram[(nodes, edges)]
        except IndexError:
            exceptions.append((nodes, edges, histogram[(nodes, edges)]))
    if exceptions:
        print(
            "Not visualised:", sum([i[2] for i in exceptions]) / total_count * 100, "%"
        )
    fig = plt.figure(dpi=200)
    im = plt.imshow(image, cmap=plt.cm.hot_r)
    cb = plt.colorbar(shrink=0.5)
    cb.set_label("Number of samples")
    plt.xlabel("Number of Edges")
    plt.ylabel("Number of Nodes")
    plt.title("Dataset: " + key)
    fig.savefig(figure_directory / f"{key}_dual_histogram.png", bbox_inches="tight")


for key in ogb_histograms:
    if not key == "ogbg-ppa":
        print("visualizing 2D histogram:", key)
        visualize_2D_histogram(ogb_histograms[key], key, dropout=0.01)

In [ ]:
def visualize_2D_histogram_builtin(histogram, key, dropout=0.01):
    """Uses plt.hist2d to do the same as the figure above"""
    total_count = sum([histogram[(nodes, edges)] for nodes, edges in histogram])
    threshold = total_count * dropout / 100
    raw_num_nodes = [nodes for nodes, edges in histogram]
    raw_num_edges = [edges for nodes, edges in histogram]
    raw_num_weights = [weights for weights in histogram.values()]
    fig = plt.figure(dpi=200)
    im = plt.hist2d(
        raw_num_edges,
        raw_num_nodes,
        weights=raw_num_weights,
        cmap=plt.cm.hot_r,
        bins=max(raw_num_nodes),
        cmin=threshold,
    )
    plt.xlabel("Number of Edges")
    plt.ylabel("Number of Nodes")
    plt.title("Dataset: " + key)
    cb = plt.colorbar(shrink=0.5)
    cb.set_label("Number of samples")
    fig.savefig(figure_directory / f"{key}_dual_histogram_v2.png", bbox_inches="tight")


for key in ogb_histograms:
    if not key == "ogbg-ppa":
        print("visualizing 2D histogram:", key)
        visualize_2D_histogram_builtin(ogb_histograms[key], key, dropout=0.01)

In the histograms, we focus on major bins and ignore the tails to get a better and faster visualisation.
For the `ogbg-ppa` this results in an empty image.
Feel free to explore different drop out rates and the `ogbg-ppa` dataset.
You can also explore the histograms for the number of nodes or edges.

In [ ]:
def visualize_nodes_edges_histogram(histogram, key, cumulative=False):
    num_nodes = []
    num_edges = []
    for nodes, edges in histogram:
        count = histogram[(nodes, edges)]
        num_nodes.extend([nodes] * count)
        num_edges.extend([edges] * count)

    min_num_nodes = int(min(num_nodes))
    max_num_nodes = int(max(num_edges))

    plt.hist(num_nodes, max_num_nodes - min_num_nodes + 1, cumulative=cumulative)
    plt.xlabel("Number of Nodes")
    plt.ylabel("Counts")
    plt.title("Histogram of number of graph nodes for " + key)
    plt.savefig(
        "graph_packing_tutorial"
        + os.sep
        + ("cumulative" if cumulative else "")
        + key
        + "_nodes_histogram.png",
        bbox_inches="tight",
    )
    plt.show()

    min_num_edges = int(min(num_edges))
    max_num_edges = int(max(num_edges))

    plt.hist(
        num_edges, max_num_edges - min_num_edges + 1, cumulative=cumulative
    )  # max_num_edges-min_num_edges+1)
    plt.xlabel("Number of Edges")
    plt.ylabel("Counts")
    plt.title(f"Histogram of number of edges for " + key)
    plt.savefig(
        "graph_packing_tutorial"
        + os.sep
        + ("cumulative" if cumulative else "")
        + key
        + "_edges_histogram.png",
        bbox_inches="tight",
    )
    plt.show()


for key in ogb_histograms:
    print("visualizing histograms: ", key)
    visualize_nodes_edges_histogram(ogb_histograms[key], key, cumulative=False)

From the visualizations, it is evident that most graphs are much smaller than the largest graph in the dataset: this means that, for a batch size of 1, a major portion of the data will be padding.

For larger batch sizes, data usually gets combined which on average may lead to a better distribution, however, the size of a batch in theory can vary as much as we have variance between individual graphs. To minimise padding, it makes sense to batch graphs that result in a constant size of nodes and edges when combined.

## Tuple Packing

To minimise the amount of padding needed to process a dataset we develop a heuristic which assigns a "sorting priority" to each graph and we will then assemble *packs* based on this priority.
This heuristic decides which graphs to tackle first as well as which graphs to prefer when filling empty spots:
a graph with a higher priority should be assigned to a *pack* before a graph with a lower priority. 
There is a variety of possibilities for this heuristic. We provide the most intuitive ones.

In [ ]:
heuristics = {
    "prod": lambda x, y: int(x * y),
    "sum": lambda x, y: int(x + y),
    "max": lambda x, y: max(x, y),
    "min": lambda x, y: max(x, y),
    "node": lambda x, y: int(x),
    "edge": lambda x, y: int(y),
}

Now, we need some fast algorithm that decides which graphs to combine based on the heuristic.

Remember that the number of graphs can be large, hence, we have to operate on the histogram to get a solution fast.

Our approach is at its core to sort the histogram by the heuristic and iterate over it.
At the beginning as well if we can't find any pack, where we can add the graph and still meet the size constraints, we start a new pack with the current graph.
A pack describes which tuple sizes get combined and how many of these combinations are obtained.
For each pack, we calculate how much space is left in each component and apply the heuristic for sorting. 
If multiple packs would fit a new incoming graph, we add it to the one with the smallest heuristic. This way, we get a near optimal fit.


In [ ]:
def pack_using_dlpfhp(
    histogram,
    max_tuple_length,
    max_tuples_per_pack,
    heuristic=lambda x, y: int(x * y),
    verbose=True,
):
    """Dual Longest-pack-first histogram-packing algorithm.

    Arguments:
        histogram Dict[Tuple[int, int], int]: The histogram of the dataset, it maps
            pairs of node and edge numbers to the number of graphs which match this
            specific size in the dataset.
        max_tuple_length (Tuple[int, int]): A pair that describes the maximum size of
            the container for each component that must be filled with
            packing. In this example this is a maximum number of nodes or edges.
        max_tuples_per_pack (int | Literal["max"]): This integer parameter limits how
                many tuples/graphs can be combined. If using "max", no limit on packs is
                set, which in some cases can slow down the packing algorithm drastically.
        heuristic (Callable[int, int]): A function which calculates the priority heuristic
            from the histogram key.
    """
    # heuristic assignment
    heuristic_data_list = [
        (heuristic(a, b), a, b, histogram[(a, b)]) for a, b in histogram
    ]
    heuristic_data_list.sort()
    heuristic_data_list.reverse()
    data_list = heuristic_data_list
    max_a, max_b = max_tuple_length[0], max_tuple_length[1]
    max_size = heuristic(max_a, max_b)
    if max_tuples_per_pack == "max":
        max_tuples_per_pack = min(max_tuple_length)
    # Initialize main strategy data dictionary.
    # The key indicates how much space is left.
    # The value is a list of tuples, consisting of counts and respective packs/tuples.
    tmp_strategies_per_length = defaultdict(list)
    strategies_per_length = defaultdict(list)
    for i in range(len(data_list)):  # distribute each bin of histogram
        size, len_a, len_b, n_sequences_to_bin = data_list[i]
        left_size = heuristic(max_a - len_a, max_b - len_b)
        offset = 0  # smallest possible offset for perfect fit
        while n_sequences_to_bin > 0:
            keys = [key for key in tmp_strategies_per_length if key >= size + offset]
            if not keys:
                offset = max_size + 1
            else:
                offset = min(keys) - size
            if (size + offset) in tmp_strategies_per_length:
                for i in range(len(tmp_strategies_per_length[size + offset])):
                    lens_a, lens_b, n_sequences_to_pack = tmp_strategies_per_length[
                        size + offset
                    ][i]
                    if (len_a + sum(lens_a)) <= max_a and (
                        len_b + sum(lens_b)
                    ) <= max_b:
                        tmp_strategies_per_length[size + offset].pop(i)
                        new_lens_a = lens_a.copy()
                        new_lens_a.append(len_a)
                        new_lens_b = lens_b.copy()
                        new_lens_b.append(len_b)
                        new_size = heuristic(
                            max_a - sum(new_lens_a), max_b - sum(new_lens_b)
                        )
                        new_count = min(n_sequences_to_pack, n_sequences_to_bin)
                        # adjust strategies
                        if n_sequences_to_pack > new_count:
                            tmp_strategies_per_length[size + offset].append(
                                (lens_a, lens_b, n_sequences_to_pack - new_count)
                            )
                        if not tmp_strategies_per_length[size + offset]:
                            tmp_strategies_per_length.pop(size + offset)
                        if new_size == 0 or max_tuples_per_pack == len(new_lens_a):
                            strategies_per_length[0].append(
                                (new_lens_a, new_lens_b, new_count)
                            )
                        else:
                            tmp_strategies_per_length[new_size].append(
                                (new_lens_a, new_lens_b, new_count)
                            )
                        n_sequences_to_bin -= new_count
                        offset = 0
                        break
            offset += 1
            if offset + size > max_size:
                new_size = heuristic(max_a - len_a, max_b - len_b)
                if new_size == 0 or max_tuples_per_pack == 1:
                    strategies_per_length[new_size].append(
                        ([len_a], [len_b], n_sequences_to_bin)
                    )
                else:
                    tmp_strategies_per_length[new_size].append(
                        ([len_a], [len_b], n_sequences_to_bin)
                    )
                n_sequences_to_bin = 0
                break
    # merge all strategies
    for key in tmp_strategies_per_length:
        strategies_per_length[key].extend(tmp_strategies_per_length[key])
    # flatten strategies dictionary
    strategy_set = []
    strategy_repeat_count = []
    sum_lens_a, sum_lens_b = [], []
    for key in strategies_per_length:
        for lens_a, lens_b, n_sequences_to_pack in strategies_per_length[key]:
            strategy_set.append((lens_a, lens_b))
            strategy_repeat_count.append(n_sequences_to_pack)
            sum_lens_a.append(sum(lens_a))
            sum_lens_b.append(sum(lens_b))
    if not (max_a == max(sum_lens_a) and max_b == max(sum_lens_b)):
        if verbose:
            print(
                "max discrepancy, reducing sequence length",
                max_a,
                max(sum_lens_a),
                max_b,
                max(sum_lens_b),
            )
        max_a, max_b = max(sum_lens_a), max(sum_lens_b)
    # efficiency calculation
    empty_tokens_a = int(
        sum(
            [
                count * (max_a - sum(pack_a))
                for count, (pack_a, pack_b) in zip(strategy_repeat_count, strategy_set)
            ]
        )
    )
    empty_tokens_b = int(
        sum(
            [
                count * (max_b - sum(pack_b))
                for count, (pack_a, pack_b) in zip(strategy_repeat_count, strategy_set)
            ]
        )
    )
    packs = int(sum(strategy_repeat_count))
    total_tokens_a, total_tokens_b = int(max_a * packs), int(max_b * packs)
    token_efficiency = (
        100 - empty_tokens_a / total_tokens_a * 100,
        100 - empty_tokens_b / total_tokens_b * 100,
    )
    return strategy_set, np.array(strategy_repeat_count), token_efficiency

Note that the code is written for tuples with two components like the number of nodes and edges but can easily be extended to more components.

The packing function calculates the "token efficiency" of the heuristic: it is the ratio of true data over the size of the padded data; larger numbers are better.

Now let's try a few heuristics to get a better picture.

In [ ]:
max_items_per_pack = 256
# other possibilities: "max", 1
heuristic = lambda x, y: int(x * y)

for key in ogb_histograms:
    # "ogbg-ppa" processing takes too long
    if key == "ogbg-ppa":
        continue
    print("----------------------------------------------------")
    print(f"Token efficiencies for '{key}' with:")
    print(f"   - maximum container sizes={ogbg_mtl_dict[key]}")
    print(f"   - max number of graphs per pack={max_items_per_pack}")
    print("  strategy, nodes, edges\n ------------------------")
    for heuristic in heuristics:
        _, _, token_efficiency = pack_using_dlpfhp(
            ogb_histograms[key],
            ogbg_mtl_dict[key],
            max_items_per_pack,
            heuristic=heuristics[heuristic],
        )
        print(f"  {heuristic}, {token_efficiency[0]:.2f}%, {token_efficiency[1]:.2f}%")

After comparing `max_items_per_pack=1` and `max_items_per_pack=256`, you will see that the efficiency improves drastically. You can also use the first case to get an estimate of the speedup you can achieve by combining graphs.

## Tuning the pack sizes

The number of tokens contained in each pack can be changed with the `max_tuple_length` argument.
It is an important parameter which can be tuned to improve the token efficiency of the packed dataset.
While the minimum size of the packs must be at least the size of the largest graph in the unpacked dataset, it can be made larger to allow more graphs to be assembled together for a higher token efficiency.

We will explore the effect of the `max_tuple_length` argument on the token efficiency when packing the `"ogbg-pcqm4mv2"` dataset.

For each parameter we collect the "node efficiency" and the "edge efficiency" which is the ratio of useful tokens over the total space for tokens in the packed dataset. These metrics give a measure of how much compute will be wasted on padding data.


In [ ]:
from tqdm import tqdm

dataset_to_analyse = "ogbg-pcqm4mv2"

max_nodes, max_edges = ogbg_mtl_dict[dataset_to_analyse]
histogram = ogb_histograms[dataset_to_analyse]
results = []
for pack_size_a in tqdm(range(max_nodes, 3 * max_nodes, 1)):
    for pack_size_b in range(max_edges, 3 * max_edges, 2):
        strategy_set, strategy_repeat_count, eff = pack_using_dlpfhp(
            histogram,
            max_tuple_length=[pack_size_a, pack_size_b],
            max_tuples_per_pack="max",
            heuristic=lambda x, y: x * y,
            verbose=False,
        )
        results.append(((pack_size_a, pack_size_b), eff))

To compare our parameters we calculate the harmonic mean of the node and edge efficiencies, giving us a single metric representing the performance of a parameter combination.
Let's sort our results to find the best parameter combinations.

In [ ]:
import pandas as pd

harmonic_efficiency = [
    (2 / (1 / node_eff + 1 / edge_eff), node_eff, edge_eff, *sizes)
    for sizes, (node_eff, edge_eff) in results
]
harmonic_efficiency.sort()
harmonic_efficiency.reverse()
efficiency_df = pd.DataFrame(
    harmonic_efficiency,
    columns=[
        "harmonic_eff",
        "node_eff",
        "edge_eff",
        "node_pack_size",
        "edge_pack_size",
    ],
)
efficiency_df.set_index(keys=["node_pack_size", "edge_pack_size"], inplace=True)
efficiency_df.head()

Now let us compare to the results for the minimal value of pack size:

In [ ]:
efficiency_df.loc[[ogbg_mtl_dict[dataset_to_analyse]], :]

To gain a better understanding of the distribution of where efficient packing parameters cluster, let us visualize the harmonic mean of the efficiencies for all combinations of parameters.

In [ ]:
import numpy

result_image = numpy.zeros([max_edges, 2 * max_nodes])
edges = []
nodes = []

for (n, e), d in results:
    result_image[(e - max_edges) // 2][n - max_nodes] = max(
        50, 2 / (1 / d[0] + 1 / d[1])
    )
    edges.append(e)
    nodes.append(n)


fig = plt.figure(dpi=300)
extent = None
extent = np.min(edges), np.max(edges), np.max(nodes), np.min(nodes)
im = plt.imshow(result_image.T, cmap=plt.cm.hot_r, extent=extent)
cb = plt.colorbar(shrink=0.5)
cb.set_label("Harmonic Efficiency (%)")
plt.xlabel("Number of Edges")
plt.ylabel("Number of Nodes")
fig.savefig(
    "graph_packing_tutorial" + os.sep + "PCQM4Mv2_tuple_pack_performance.png",
    bbox_inches="tight",
)

We can see that the best harmonic mean of the packing efficiency on the nodes and edges is obtained at 30 nodes and 62 edges with a value of (98.6%, 99.0%).
It is close to perfect, and much better than the 70% achieved when using the maximum sizes found in the ogbg-pcqm4mv2 dataset (20 nodes and 56 edges).
We also get good results for 45 nodes and 96 edges (94.9%, 98.4%). Having multiple efficient packing parameter combinations is useful as it lets us tradeoff some packing efficiency with compute efficiency, as bigger packs may allow us to use the IPUs more efficiently, while smaller packs can help a model fit on fewer IPUs. The good results are mostly centred close to the diagonal where there are twice as many edges as nodes.

## Conclusion

This tutorial is a companion to the paper "Tuple Packing: Efficient Batching of Small Graphs in Graph Neural Networks" available on [arxiv](https://arxiv.org/abs/2209.06354).
It worked through the calculation of efficient packs for a range of datasets in the OGB competition.
We showed how histograms can be used to efficiently calculate heuristics and assemble optimised packs of graphs for processing in GNNs.
We also analysed the importance of pack size on the efficiency of the packed dataset.

If you like this tutorial and use the code, please cite our paper.